In [1]:
# Install packages from requirements.txt

# %pip install -r requirements.txt

# imports

In [2]:
# Set the working directory and adjust the Python path
import os
import sys

# Set the working directory to the src directory
os.chdir('./')
sys.path.append(os.getcwd())

from scripts.utils.generators import *
from scripts.test.test_graph_generators import *
from configuration.algorithm_settings import settings

import pprint


# get graph map

In [38]:
size=10
fixed_weight=0.1

map_graph = generate_square_city_graph(size,fixed_weight)

In [39]:
buses_graph = generate_bus_line_square_city(size,fixed_weight)

In [40]:
map_graph = merge_bus_and_map_graph(map_graph,buses_graph)

# run ACO algorithms

## simple ACO

In [73]:
import numpy as np
from scripts.utils.roulette_selection import roulette_wheel_selection

def ant_solution_ACO(graph_map: dict, pheromone_graph:dict, start_node:int, end_node:int, heuristic_weight:float, pheromone_weight:float):

    solution_path = [start_node]
    solution_cost = 0

    while solution_path[-1] != end_node:
        current_node = solution_path[-1]
        neighbors = np.array(graph_map["connections"][current_node])
        neighbors_weights = np.array(graph_map["weights"][current_node])
        neighbors_pheromones = np.array(pheromone_graph[current_node])

        filter_visited_nodes_mask = ~np.isin(neighbors, solution_path)
        neighbors = neighbors[filter_visited_nodes_mask]
        neighbors_weights = neighbors_weights[filter_visited_nodes_mask]
        neighbors_pheromones = neighbors_pheromones[filter_visited_nodes_mask]

        if len(neighbors) == 0:
            solution_path.append(float('inf'))  # The ant is lost. Stop the search
            break

        # Calculate probabilities for moving to the next node
        pheromone_values = neighbors_pheromones ** heuristic_weight
        heuristic_values = (1.0 / neighbors_weights) ** pheromone_weight
        sum_values = np.sum(pheromone_values * heuristic_values)
        probabilities = (pheromone_values * heuristic_values) / sum_values

        # Select the next node based on the roulette wheel selection
        next_node_index = roulette_wheel_selection(probabilities)
        solution_path.append(neighbors[next_node_index-1])

    if solution_path[-1] != float('inf'):  # If the ant is not lost, return the path and calculate the total cost
        for i in range(len(solution_path) - 1):
          neighbor_selected_index = graph_map["connections"][solution_path[i]].index(solution_path[i + 1])
          solution_cost += graph_map["weights"][solution_path[i]][neighbor_selected_index]
    else:
        solution_cost = float('inf')

    return solution_path,solution_cost

In [74]:
from time import time
from collections import Counter
from scripts.utils.generators import generate_pheromone_map

def ACO(graph_map, start_point, end_point, ants_number, evaporation_rate, initial_pheromone_lvl, heuristic_weight, pheromone_weight):

    pheromone_graph = generate_pheromone_map(graph_map,initial_pheromone_lvl)
    routes = [None] * ants_number  # Paths taken by each ant
    distances = np.zeros(ants_number)  # To sort solutions from best to worst

    epochs = 0
    counter = 0

    start_time = time()
    while counter < ants_number:
        # Each ant makes its journey
        for ant in range(ants_number):
            path_found,path_distance = ant_solution_ACO(graph_map, pheromone_graph, start_point, end_point, heuristic_weight, pheromone_weight)
            routes[ant] = path_found
            distances[ant] = path_distance

        # Global pheromone evaporation
        for key in pheromone_graph:
            pheromone_graph[key] *= (1 - evaporation_rate)

        # Global pheromone deposition
        for ant in range(ants_number):
            if distances[ant] != np.inf:
                for i in range(len(routes[ant]) - 2):
                    current_route_node = routes[ant][i]
                    next_route_node = routes[ant][i+1]
                    indx_next_node = graph_map["connections"][current_route_node].index(next_route_node)
                    pheromone_graph[current_route_node][indx_next_node] += 1 / distances[ant]

        # Analyze algorithm termination criteria
        number_of_solutions = distances[distances != np.inf].size  # Paths of lost ants don't count
        if number_of_solutions > 0:
            _, counter = Counter(distances[distances != np.inf]).most_common(1)[0]

        epochs += 1

    # Return the optimal path
    path = routes[0][:-1]
    cost = distances[0]
    total_time = time() - start_time

    return path, cost, total_time, epochs

In [75]:
start_node = 4
end_node = 52
num_ants = settings["ants"]
evaporation_rate =settings["evaporation_rate"]
initial_pheromone_lvl =settings["f_ini"]
heuristic_weight=settings["alfa"]
pheromone_weight=settings["beta"]

solution_path,cost,time,epochs = ACO(map_graph,start_node,end_node,num_ants,evaporation_rate,initial_pheromone_lvl,heuristic_weight,pheromone_weight)

print(solution_path)
print(cost)
print(time)
print(epochs)

[4, 5, 15, 25, 35, 1035, 1045, 45, 55, 54, 44, 43, 42, 41, 51, 61, 62, 63, 53]
1.7950000000000006
2.7326159477233887
24
